In [ ]:
%matplotlib inline
import subprocess
import yaml
import scipy.misc
import re
import numpy as np
from types import SimpleNamespace
from matplotlib import pyplot

subprocess.call(['latexmk', '-pdf', 'test.tex'])

In [ ]:
with open('tussentoets2016-6.yml') as f:
    data = yaml.load(f)

In [ ]:
widgets = data['widgets']

widgets = [SimpleNamespace(**{key: (int(300*eval(value)/2**16/72) if isinstance(value, str) else value)
                         for key, value in widget.items()}) for widget in widgets.values() if widget['page'] == 1]

In [ ]:
import os
os.chdir('test_exam/')

In [ ]:
!convert -density 300 tussentoets2016-6.pdf test_page_%02d.png

In [ ]:
images = []
for image in os.listdir():
    if image.endswith('.png'):
        images.append(scipy.misc.imread(image, flatten=True))

In [ ]:
qr1

In [ ]:
im = images[0]


mask = np.zeros_like(im)
for widget in widgets:
    mask[-widget.top:-widget.bottom+15, widget.left-+15:widget.right] = -255
pyplot.figure(figsize=(20, 20))
pyplot.imshow(im + mask, cmap='gray', interpolation='bilinear')

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer

# Open a PDF file.
fp = open('20170206145950.pdf', 'rb')

# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)

# Create a PDF document object that stores the document structure.
# Password for initialization as 2nd parameter
document = PDFDocument(parser)

# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a PDF device object.
device = PDFDevice(rsrcmgr)

# BEGIN LAYOUT ANALYSIS
# Set parameters for analysis.
laparams = LAParams()

# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)

def parse_obj(lt_objs):

    # loop over the object list
    for obj in lt_objs:

        # if it's a textbox, print text and location
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            print("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text().replace('\n', '_')))

        # if it's a container, recurse
        elif isinstance(obj, pdfminer.layout.LTFigure):
            parse_obj(obj._objs)

# loop over all pages in the document
for page in PDFPage.create_pages(document):

    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()

    # extract text from this object
    parse_obj(layout._objs)

In [ ]:
import pdfminer

In [ ]:
fp = open('20170206145950.pdf', 'rb')

parser = PDFParser(fp)

In [ ]:
fp.close()

In [ ]:
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine

fp = open('20170206145950.pdf', 'rb')
parser = PDFParser(fp)
doc = PDFDocument()
parser.set_document(doc)
doc.set_parser(parser)
doc.initialize('')
rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
for page in doc.get_pages():
    interpreter.process_page(page)
    layout = device.get_result()
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            print(lt_obj.get_text())
